In [4]:
import json
import pandas as pd
import torch
cuda0 = torch.device('cuda:0')

In [ ]:
repo_teams = {}
with open('repo_profiles_new.json') as rj:
    for rl in rj.readlines():
        line = rl.split('\t')
        repo = line[0]
        profile = json.loads(line[1])
        repo_teams[repo] = profile['teams']

repo_graph = []
repos = []
cnt = 0
r = 3000
with open('repo_graph_sim.json') as rj:
    for rl in rj.readlines():
        cnt +=1
        if cnt > r:
            break
        line = rl.split('\t')
        repo = line[0]
        repos.append(repo)
        repo_graph.append(json.loads(line[1])[:r])
print(len(repo_graph))

In [2]:
def sort_to_k(ary,k,key=lambda x:x,reversed=False):
    k = min(k,len(ary))
    for i in range(k):
        for j in range(len(ary)-1-i):
            if not reversed:
                if key(ary[len(ary)-1-j]) < key(ary[len(ary)-2-j]):
                    ary[len(ary)-1-j],ary[len(ary)-2-j] = ary[len(ary)-2-j],ary[len(ary)-1-j]
            else:
                if key(ary[len(ary)-1-j]) > key(ary[len(ary)-2-j]):
                    ary[len(ary)-1-j],ary[len(ary)-2-j] = ary[len(ary)-2-j],ary[len(ary)-1-j]
    return ary

In [3]:
def recommend_random_walk(repo_graph,repos,repo_teams,device,steps,k):
    cnt = 0
    transfer = 1/torch.tensor(repo_graph,requires_grad=False,dtype=torch.float32,device=device)
    transfer /= transfer.sum(0)
    for i in range(steps-1):
        transfer = transfer.matmul(transfer)
    for repo in repos:
        print(cnt)
        if cnt>5:
            break
        page_rank = torch.zeros(len(repos),requires_grad=False,device=device)
        page_rank[cnt] = 1
        page_rank = page_rank.matmul(transfer)
        page_rank = page_rank.cpu().numpy()
        team_rank = {}
        for i,p_r in enumerate(page_rank):
            for team in repo_teams[repos[i]]:
                if not team in team_rank:
                    team_rank[team] = 0
                team_rank[team] += p_r
        teams = sort_to_k(list(team_rank.keys()),k,key=lambda i:team_rank[i])
        print(repo)
        for i in range(k):
            print(teams[i])
        cnt += 1

In [4]:
steps = 10
k=5

recommend_random_walk(repo_graph,repos,repo_teams,cuda0,steps,k)

0
Starbound-Dev/starbound-dev
["vincenthz", "dcoutts", "erikd", "arybczak", "int-index", "kantp", "volhovm", "avieth", "m-schmidt", "vmchale", "pva701", "enzo1982", "brunjlar", "adinapoli-iohk"]
["brillout", "lsycxyj", "sagalbot", "ktquez", "SidKwok"]
["sergeche", "shinnn", "yisibl", "morishitter", "seka", "fliptheweb", "hudochenkov", "peterramsing", "mrmlnc", "gucong3000", "tylersnyder", "Gingernaut"]
["yyx990803", "oaleynik", "Austio", "miljan-aleksic", "mondaychen", "sunabozu"]
["KissDaniGH", "chfsx", "corro", "simoor", "raimann", "aarsenij", "d3r1w", "bogenschlag", "lechimp-p", "bseglias"]
1
betheluniversity/Tinker
["ssayer", "eedrummer", "rdingwell", "Xodarap", "mlos100", "ttaylor249", "jmazella"]
["mde", "tbjers", "phanect"]
["dougwilson", "alvarotrigo", "jonathanong", "tj", "gabeio", "adrianblynch", "SheetJSDev", "Jokero", "jancarloviray", "calebboyd", "seangarner", "miletbaker", "WORMSS", "slaskis", "palanik", "franciscop", "nooks", "IShinji", "federomero", "ismailarilik"]
["ol

In [5]:
r_g = []
repos = []
cnt = 0
r = 3000
steps = 10
k=5
with open('repo_graph_ref.json') as rj:
    for rl in rj.readlines():
        cnt +=1
        if cnt > r:
            break
        line = rl.split('\t')
        repo = line[0]
        repos.append(repo)
        r_g.append(json.loads(line[1]))

repo_graph = []
for g in r_g:
    repo_graph.append([0]*len(repos))
    for i in g:
        if int(i) < r:
            repo_graph[-1][int(i)] = g[i]

for i in range(len(repo_graph)):
    for j in range(i):
        repo_graph[i][j] = repo_graph[i][j]+repo_graph[j][i]
        repo_graph[j][i] = repo_graph[i][j]
    repo_graph[i][i] = 0

In [6]:
recommend_random_walk(repo_graph,repos,repo_teams,cuda0,steps,k)

0
Starbound-Dev/starbound-dev
["simi", "prokopsimek", "CJBrew", "thomasdarde", "parndt", "bricesanchez", "robyurkowski", "johanb", "mattherick", "giedriusr", "zmoazeni", "anitagraham", "dalpo", "Spaceghost", "Kagetsuki", "ghoppe", "gwagener", "tomash", "meleyal", "danielfone", "kevin-jj", "ur5us", "awagener", "jess", "shioyama", "kgautreaux", "snake66", "ridiculous", "joshmcarthur", "snmgian", "ybart", "sjaveed", "simmerz", "novikserg", "andreazaupa", "Znow", "Bramjetten", "shivabhusal", "mabras", "sjoulbak", "peresleguine", "NicholasKirchner", "brigriffin", "shanser"]
["simi", "JonRowe", "parndt", "route", "scarfacedeb", "pat", "bsodmike"]
["justsml", "lord", "ecaron", "tariqislam", "halfcrazy", "anfedorov", "rhruiz", "jaythomas", "Jameskmonger", "jamesadarich", "PotHix", "MEGApixel23", "evilstickman", "eliat123", "mhrabiee"]
["dbkaplun", "wankdanker", "lee-houghton", "addaleax", "styfle", "Flarna", "MoonBall"]
["bnoordhuis", "targos", "thefourtheye", "joransiu", "SirR4T", "tflanagan"

In [6]:
t_g = []
teams = []
team_ind = {}
cnt = 0
r = 3000
steps = 10
k=5
with open('team_graph.json') as rj:
    for rl in rj.readlines():
        cnt +=1
        if cnt > r:
            break
        line = rl.split('\t')
        team = line[0]
        teams.append(team)
        team_ind[team] = len(teams)-1
        t_g.append(json.loads(line[1]))
repos = []
repo_teams = {}
cnt = 0
with open('repo_profiles_new.json') as rj:
    for rl in rj.readlines():
        if cnt > r:
            break
        line = rl.split('\t')
        repo = line[0]
        profile = json.loads(line[1])
        repos.append(repo)
        repo_teams[repo] = profile['teams']

team_graph = []
for g in t_g:
    team_graph.append([0]*len(teams))
    for i in g:
        if int(i) < r:
            team_graph[-1][int(i)] = g[i]

for i in range(len(team_graph)):
    for j in range(i):
        team_graph[i][j] = team_graph[i][j]+team_graph[j][i]
        team_graph[j][i] = team_graph[i][j]
    team_graph[i][i] = 0
print(len(repos))
print(team_graph[0][:3])

14447
[0, 0, 0]


In [22]:
def recommend_social(team_graph,repos,teams,team_ind,device,steps,k,alpha):
    cnt = 0
    transfer = torch.tensor(team_graph,requires_grad=False,dtype=torch.float32,device=device)
    transfer /= transfer.sum(0)
    transfer[transfer!=transfer] = 0
    print(transfer[9].sum())
    for i in range(steps-1):
        transfer = transfer.matmul(transfer)
    for repo in repos:
        print(cnt)
        if cnt>5:
            break
        page_rank = torch.zeros(len(teams),requires_grad=False,device=cuda0)
        for team in repo_teams[repo]:
            if team in team_ind:
                page_rank[team_ind[team]] = 1/len(repo_teams[repo])
        for i in range(steps):
            page_rank = alpha*page_rank.matmul(transfer) + (1-alpha)/len(teams)
        page_rank = page_rank.cpu().numpy()
        print(page_rank[0])

        tms = sort_to_k(list(range(len(teams))),k,key=lambda i:page_rank[i],reversed=True)
        print(repo)
        for i in range(k):
            print(json.dumps((teams[tms[i]],page_rank[tms[i]].item())))
        cnt += 1

In [23]:
alpha=0.95
recommend_social(team_graph,repos,teams,team_ind,cuda0,steps,k,alpha)

tensor(1., device='cuda:0')
0
0.00013372499
Starbound-Dev/starbound-dev
["[\"sskinner\", \"heather-lo\", \"Perrinek\", \"jcaddel\", \"quikkian-ua-devops\", \"tounou\"]", 0.000133761772303842]
["[\"cniesen\", \"ssaripel\", \"coreypedersen\"]", 0.00013376175775192678]
["[\"sonamuthu\", \"shahess\"]", 0.00013376175775192678]
["[\"petitrenaudseb\", \"AIAitesla\"]", 0.0001337611029157415]
["[\"yichen88\", \"mathbagu\"]", 0.0001337611029157415]
1
0.00013372499
betheluniversity/Tinker
["[\"sskinner\", \"heather-lo\", \"Perrinek\", \"jcaddel\", \"quikkian-ua-devops\", \"tounou\"]", 0.000133761772303842]
["[\"cniesen\", \"ssaripel\", \"coreypedersen\"]", 0.00013376175775192678]
["[\"sonamuthu\", \"shahess\"]", 0.00013376175775192678]
["[\"petitrenaudseb\", \"AIAitesla\"]", 0.0001337611029157415]
["[\"yichen88\", \"mathbagu\"]", 0.0001337611029157415]
2
0.00013372499
refinery/refinerycms-news
["[\"sskinner\", \"heather-lo\", \"Perrinek\", \"jcaddel\", \"quikkian-ua-devops\", \"tounou\"]", 0.0001